# Caffe2 Introduction

References
* https://caffe2.ai/docs/intro-tutorial.html

In [1]:
from caffe2.python import workspace, model_helper
import numpy as np

## Blobs and Workspace, Tensors

In [2]:
# Create random tensor of three dimensions
x = np.random.rand(4, 3, 2)
print(x)
print(x.shape)

workspace.FeedBlob("my_x", x)

x2 = workspace.FetchBlob("my_x")
print(x2)

[[[0.20067819 0.02449249]
  [0.99203175 0.59753587]
  [0.64125494 0.64804213]]

 [[0.83286782 0.7166435 ]
  [0.90717252 0.35607617]
  [0.61197566 0.06457378]]

 [[0.18070491 0.465027  ]
  [0.43689523 0.03734451]
  [0.75650208 0.06555338]]

 [[0.19843814 0.33430901]
  [0.41172729 0.82772944]
  [0.98035546 0.81931098]]]
(4, 3, 2)
[[[0.20067819 0.02449249]
  [0.99203175 0.59753587]
  [0.64125494 0.64804213]]

 [[0.83286782 0.7166435 ]
  [0.90717252 0.35607617]
  [0.61197566 0.06457378]]

 [[0.18070491 0.465027  ]
  [0.43689523 0.03734451]
  [0.75650208 0.06555338]]

 [[0.19843814 0.33430901]
  [0.41172729 0.82772944]
  [0.98035546 0.81931098]]]


## Nets and Operators

### 1. Model defintion

In [3]:
# Create the input data
data = np.random.rand(16, 100).astype(np.float32)

# Create labels for the data as integers [0, 9].
label = (np.random.rand(16) * 10).astype(np.int32)

workspace.FeedBlob("data", data)
workspace.FeedBlob("label", label)

True

In [4]:
# Create model using a model helper
m = model_helper.ModelHelper(name="my first net")

In [5]:
weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10, 100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10, ])

In [6]:
fc_1 = m.net.FC(["data", "fc_w", "fc_b"], "fc1")
pred = m.net.Sigmoid(fc_1, "pred")
softmax, loss = m.net.SoftmaxWithLoss([pred, "label"], ["softmax", "loss"])
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label"



### 2. Executing

In [7]:
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net)

True

In [8]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [9]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.09729729 0.10235403 0.10384981 0.07824369 0.10053563 0.09581183
  0.10055164 0.11575191 0.10560181 0.10000231]
 [0.09829728 0.09779312 0.10761204 0.09588309 0.1053839  0.10047052
  0.09579735 0.10051629 0.08623596 0.11201049]
 [0.10254492 0.1000658  0.12236757 0.09416427 0.09513773 0.10812505
  0.08081137 0.0962331  0.09884302 0.10170718]
 [0.09716035 0.115008   0.10897743 0.08455004 0.10010631 0.10044482
  0.0822323  0.1231948  0.09240324 0.09592273]
 [0.09660865 0.09433345 0.10561669 0.08835956 0.10377239 0.10553793
  0.09026746 0.11326636 0.09641717 0.10582037]
 [0.08974919 0.10719682 0.1073035  0.10885403 0.1102971  0.09536776
  0.08648033 0.10708412 0.0910176  0.09664951]
 [0.09131484 0.09692156 0.11508288 0.08642853 0.10682766 0.10219012
  0.09892079 0.10925152 0.09625351 0.09680849]
 [0.09953253 0.10842919 0.11357266 0.09210159 0.09289826 0.09680344
  0.09043171 0.10663782 0.09215952 0.10743321]
 [0.08794647 0.09709716 0.10826972 0.09107433 0.11456678 0.1017866
  0.09554264 

### 3. Backward pass

In [10]:
m.AddGradientOperators([loss])
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net, overwrite=True)

True

In [11]:
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label

In [12]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [13]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.10741661 0.09018537 0.11096521 0.08871864 0.10243134 0.11817458
  0.10142485 0.10307746 0.08793548 0.08967046]
 [0.1056387  0.09031768 0.12618302 0.08451276 0.09144166 0.11505408
  0.09871291 0.1266204  0.07959548 0.08192331]
 [0.09873072 0.08720124 0.11509904 0.1014495  0.10563331 0.11236385
  0.10088694 0.10559433 0.0861948  0.08684622]
 [0.09674465 0.08060104 0.12065359 0.09467604 0.10162571 0.12792525
  0.08339298 0.1182773  0.07969061 0.09641278]
 [0.09547433 0.08987777 0.11345009 0.09911485 0.10847156 0.10083742
  0.11175764 0.1100653  0.08587397 0.08507708]
 [0.09998269 0.10177454 0.10350779 0.09599867 0.09265493 0.10906152
  0.11581226 0.1076311  0.08478606 0.08879045]
 [0.1169591  0.09504047 0.1028205  0.08593046 0.09365419 0.10859966
  0.10941872 0.10325938 0.09297835 0.09133919]
 [0.11289662 0.08737186 0.12686177 0.10058434 0.09848739 0.11104294
  0.08647525 0.09786285 0.08980101 0.088616  ]
 [0.11198892 0.08604252 0.10193547 0.09065288 0.09573968 0.11635017
  0.11265753